In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


View.ipynb viewing functions

In [18]:
import matplotlib.pyplot as plt

def image_normalization(arr):
    return (arr - arr.min())/(arr.max()-arr.min())

def disable_ax_ticks(ax):
    ax.set_xticks([])
    ax.set_xticks([], minor=True)
    ax.set_yticks([])
    ax.set_yticks([], minor=True)

def show_mnist_examples(x, y):
    fig = plt.figure(constrained_layout=True,figsize=(12,9), dpi=100)
    gs = fig.add_gridspec(3,4)
    main_ax = fig.add_subplot(gs[:3,:3])
    fig.suptitle(y)
    #main_ax.imshow(np.moveaxis(x, 0, -1))# .imshow(x)
    main_ax.imshow(image_normalization(np.moveaxis(x, 0, -1)))
    disable_ax_ticks(main_ax)

    for j in range(3):
      c_ax = fig.add_subplot(gs[j,-1])
      subimage = x.copy()
      subimage[:j] = 0
      subimage[j+1:] = 0
      subimage[j] = subimage[j]-subimage[j].min()
      c_ax.imshow(image_normalization(np.moveaxis(subimage, 0, -1)))
      disable_ax_ticks(c_ax)
    plt.show()

Loading dataset

In [19]:
import numpy as np
train_X = np.load("/content/drive/MyDrive/MLCoursework/train_x.npy", allow_pickle=True)
train_y = np.load("/content/drive/MyDrive/MLCoursework/train_y.npy", allow_pickle=True)
validate_X = np.load("/content/drive/MyDrive/MLCoursework/valid_x.npy", allow_pickle=True)
validate_y = np.load("/content/drive/MyDrive/MLCoursework/valid_y.npy", allow_pickle=True)
test_x = np.load("/content/drive/MyDrive/MLCoursework/test_x.npy", allow_pickle=True)
test_y = np.load("/content/drive/MyDrive/MLCoursework/test_y.npy", allow_pickle=True)

Constants

In [20]:
num_classes = 20
input_shape = (32, 32, 3)

Importing ML libraries

In [21]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow

In [22]:
# Normalizes dataset from 0 - 131 to 0 - 1
train_X = image_normalization(train_X)
validate_X = image_normalization(validate_X)

# Pad with 0s so resolution will fit with pretrained model
pad_train_x = np.pad(train_X, [(0,0), (0, 0), (2, 2), (2, 2)], mode='constant')
pad_val_x = np.pad(validate_X, [(0,0), (0, 0), (2, 2), (2, 2)], mode='constant')

# Change to channel last ordering 
pad_train_x = np.moveaxis(pad_train_x, 1, 3)
pad_val_x = np.moveaxis(pad_val_x, 1, 3)

# One hot encoding target feature 
train_y = keras.utils.to_categorical(train_y, num_classes)
validate_y = keras.utils.to_categorical(validate_y, num_classes)

Setting up keras model

In [25]:
from tokenize import Number
from tensorflow.keras.applications.vgg16 import VGG16

# Loading base model and setting to untrainable
base_model = VGG16(weights='imagenet',
                      include_top=False, 
                      input_shape=(32, 32, 3))
base_model.trainable = False

# creating model on top
inputs = keras.Input(shape=(input_shape))
x = base_model(inputs, training=False)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.Dense(128, activation='relu')(x)
outputs = keras.layers.Dense(num_classes, activation="softmax")(x)
model = keras.Model(inputs, outputs)

# Displays model layers
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_2 (Flatten)         (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 128)               65664     
                                                                 
 dense_6 (Dense)             (None, 128)               16512     
                                                                 
 dense_7 (Dense)             (None, 20)                2580      
                                                                 
Total params: 14,799,444
Trainable params: 84,756
Non-train

Compiling and fitting model 

In [26]:
batch_size = 125
# For computation reasons I only do 1 epoch
epochs = 1

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(pad_train_x, train_y, batch_size=batch_size, epochs=epochs, validation_data=(pad_val_x, validate_y))


360/360 [==============================] - 629s 2s/step - loss: 2.6699 - accuracy: 0.1402 - val_loss: 2.5308 - val_accuracy: 0.1687


F1 Testing over dataset

In [30]:
from sklearn.metrics import classification_report
import time 

t0 = time.clock()
pad_test_x = np.pad(test_x, [(0,0), (0, 0), (2, 2), (2, 2)], mode='constant')
# Converts to channel last ordering
pad_test_x = np.moveaxis(pad_test_x, 1, -1)

# Finding model prediction probabilities 
test_pred = model.predict(pad_test_x)
# Converts probabilities to category 
test_class_pred = np.argmax(test_pred,axis=1)

# Print classification report 
print(classification_report(test_y, test_class_pred))

# time taken
t1 = time.clock() - 0
print("CPU seconds: ", t1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  after removing the cwd from sys.path.


              precision    recall  f1-score   support

           0       0.42      0.71      0.53       395
           1       0.80      0.01      0.02       423
           2       0.16      0.12      0.14       454
           3       0.00      0.00      0.00       504
           4       0.00      0.00      0.00       447
           5       0.15      0.01      0.01       544
           6       0.19      0.02      0.03       497
           7       0.05      0.13      0.07       466
           8       0.00      0.00      0.00       527
           9       0.00      0.00      0.00       534
          10       0.06      0.01      0.01       531
          11       0.05      0.08      0.06       544
          12       0.00      0.00      0.00       536
          13       0.05      0.01      0.02       506
          14       0.03      0.02      0.02       519
          15       0.00      0.00      0.00       507
          16       0.04      0.08      0.05       477
          17       0.04    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: De